# SQL Options in Spark HW

Alright let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SQLOptions').getOrCreate()
spark

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [7]:
apps = spark.read.csv('./data/googleplaystore.csv', header=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [8]:
apps.limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


As well as the schema to make sure all the column types were correctly infered

In [9]:
apps.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [10]:
from pyspark.sql.types import IntegerType, FloatType
newdf = apps.withColumn("Rating", apps["Rating"].cast(FloatType())) \
            .withColumn("Reviews", apps["Reviews"].cast(IntegerType())) \
            .withColumn("Price", apps["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [11]:
newdf.createTempView('df_apps')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [15]:
spark.sql('SELECT App, Category, Rating FROM df_apps WHERE Rating > 4.1').limit(10).toPandas()

,App,Category,Rating
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3
3,Paper flowers instructions,ART_AND_DESIGN,4.4
4,Garden Coloring Book,ART_AND_DESIGN,4.4
5,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7
6,Text on Photo - Fonteee,ART_AND_DESIGN,4.4
7,Name Art Photo Editor - Focus n Filters,ART_AND_DESIGN,4.4
8,Tattoo Name On My Photo Editor,ART_AND_DESIGN,4.2
9,Mandala Coloring Book,ART_AND_DESIGN,4.6


In [22]:
spark.sql('SELECT DISTINCT Category from df_apps').toPandas()

,Category
0,EVENTS
1,COMICS
2,SPORTS
3,WEATHER
4,VIDEO_PLAYERS
5,AUTO_AND_VEHICLES
6,PARENTING
7,ENTERTAINMENT
8,PERSONALIZATION
9,HEALTH_AND_FITNESS


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comics category and the Rating is above 4.5.

In [23]:
df_best_comic_apps = spark.sql('SELECT * FROM df_apps WHERE Category LIKE \'COMICS\' AND Rating > 4.5 ')
df_best_comic_apps.limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Manga Master - Best manga & comic reader,COMICS,4.6,24005,4.9M,"500,000+",Free,0,Adults only 18+,Comics,"July 4, 2018",1.1.7.0,4.1 and up
1,GANMA! - All original stories free of charge f...,COMICS,4.7,57106,27M,"1,000,000+",Free,0,Teen,Comics,"July 23, 2018",2.11.3,4.1 and up
2,Röhrich Werner Soundboard,COMICS,4.7,2249,32M,"500,000+",Free,0,Everyone,Comics,"November 16, 2017",1.08,4.0.3 and up
3,Unicorn Pokez - Color By Number,COMICS,4.8,516,12M,"50,000+",Free,0,Everyone,Comics;Creativity,"August 3, 2018",1.0.6,4.4 and up
4,Manga - read Thai translation,COMICS,4.6,302,2.2M,"10,000+",Free,0,Teen,Comics,"July 3, 2018",1.0,4.0 and up
5,Dragon Ball Wallpaper - Ringtones,COMICS,4.7,73,8.3M,"10,000+",Free,0,Everyone,Comics,"July 23, 2018",1.9,4.0.3 and up
6,Children's cartoons (Mithu-Mina-Raju),COMICS,4.6,279,3.3M,"100,000+",Free,0,Everyone,Comics,"December 18, 2017",1.6,4.0 and up
7,【Ranobbe complete free】 Novelba - Free app tha...,COMICS,NaN,1330,22M,"50,000+",Free,0,Everyone,Comics,"July 3, 2018",6.1.1,4.2 and up
8,Faustop Sounds,COMICS,4.7,1677,40M,"100,000+",Free,0,Everyone,Comics,"May 26, 2018",1.7.7,4.1 and up
9,Hojiboy Tojiboyev Life Hacks,COMICS,5.0,15,37M,"1,000+",Free,0,Everyone,Comics,"June 26, 2018",2.0,4.0.3 and up


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [26]:
spark.sql('SELECT Category, sum(Reviews) AS TotalReviews FROM df_apps GROUP BY Category ORDER BY TotalReviews DESC').limit(1).toPandas()

,Category,TotalReviews
0,GAME,1585422349


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [27]:
spark.sql('SELECT App, Reviews FROM df_apps ORDER BY Reviews DESC LIMIT 1').toPandas()

,App,Reviews
0,Facebook,78158306


## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [28]:
spark.sql('SELECT App FROM df_apps WHERE App LIKE \'%dating%\'').limit(10).toPandas()

,App
0,"Meet, chat & date. Free dating app - Chocolate..."
1,Friend Find: free chat + flirt dating app
2,Spine- The dating app
3,Princess Closet : Otome games free dating sim
4,happn – Local dating app


## 6. Use SQL Transformer to display how many free apps there are in this list

In [33]:
from pyspark.ml.feature import SQLTransformer

sql_free_apps = SQLTransformer(statement='SELECT count(Price) AS Free_Apps_Count FROM __THIS__ WHERE Price = 0')

sql_free_apps.transform(newdf).toPandas()

,Free_Apps_Count
0,10038


## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [36]:
sql_most_popular_genre = \
    SQLTransformer(statement='SELECT Genres, count(App) AS AppCount FROM __THIS__ GROUP BY Genres ORDER BY AppCount DESC LIMIT 1')

sql_most_popular_genre.transform(newdf).toPandas()

,Genres,AppCount
0,Tools,842


## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [38]:
spark.sql('SELECT App, Genres, Reviews FROM df_apps WHERE Genres LIKE \'Tools\' AND Reviews > 100').limit(10).toPandas()

,App,Genres,Reviews
0,Moto File Manager,Tools,38655
1,Google,Tools,8033493
2,Google Translate,Tools,5745093
3,Moto Display,Tools,18239
4,Motorola Alert,Tools,24199
5,Motorola Assist,Tools,37333
6,Cache Cleaner-DU Speed Booster (booster & clea...,Tools,12759663
7,Moto Suggestions ™,Tools,308
8,Moto Voice,Tools,33216
9,Calculator,Tools,40770


## That's all folks! Great job!